# Comparação entre bases SIH e SINASC

## Código

In [1]:
import config, data_load
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [2]:
def read_query(query_name):
	path_query = f'queries/compare/{query_name}.sql'
	with open(path_query) as f:
		query = f.read()
	return query

In [3]:
def query_result(query_name):
	query = read_query(query_name)
	df = data_load.df_query(query)
	return df

In [4]:
def query_cnes_result(query_name,
		up_bound, low_bound, cnes_pref=None):
	query = read_query(query_name)
	query = query.replace('UP_BOUND', str(up_bound))
	query = query.replace('LOW_BOUND', str(low_bound))
	if cnes_pref != None:
		increment = "\n	and CODESTAB not like '2%'"
		if cnes_pref == False:
			increment = increment.replace(' not', '')
			print(increment)
		query = query.replace('Hospital', increment)
	df = data_load.df_query(query)
	return df

## Bases originais

In [5]:
all_sih = 157033546 # query_result('count_sih')['records'][0]
all_sinasc = 74398079 # query_result('count_sinasc')['records'][0]

In [6]:
df_count = pd.DataFrame([
	['SIH', all_sih],
	['SINASC', all_sinasc]
], columns=['database', 'records'])
fig = px.bar(df_count, x='database', y='records')
fig.show()

## Bases filtradas

In [7]:
flt_sih = 16814490 # query_result('filter_sih')['records'][0]
flt_sinasc = 28197164 # query_result('filter_sinasc')['records'][0]

In [8]:
df_filter = pd.DataFrame([
	['SIH', flt_sih],
	['SINASC', flt_sinasc]
], columns=['database', 'records'])
fig = px.bar(df_filter, x='database', y='records')
fig.show()

## Bases filtradas com apenas CNES SUS

In [10]:
df_cnes = pd.read_csv(
	'data/consult/compare/cnes_sus.csv',
	dtype={'CNES':str})
df_cnes

,CNES,repeted
0,2002043,163
1,2002159,163
2,3006166,163
3,3382745,163
4,3393984,163
...,...,...
130269,9708618,1
130270,9715568,1
130271,3475840,1
130272,3933067,1


In [11]:
# df_cnes_sih = pd.concat([
# 	query_cnes_result('sus_sih', 10**6, 4200),
# 	query_cnes_result('sus_sih', 4200, 1200),
# 	query_cnes_result('sus_sih', 1200, 270),
# 	query_cnes_result('sus_sih', 270, 6),
# 	query_cnes_result('sus_sih', 6, 1),
# ])
path_cnes_sih = 'data/consult/compare/cnes_sih.csv'
# df_cnes_sih.to_csv(path_cnes_sih, index=False)
df_cnes_sih = pd.read_csv(
	path_cnes_sih,
	dtype={'CNES':str})
df_cnes_sih

,CNES,records
0,0002232,101955
1,3151794,75209
2,2323397,74417
3,2311682,64833
4,0026794,63700
...,...,...
4144,7320175,1
4145,7603029,1
4146,7621442,1
4147,7704364,1


In [12]:
# df_cnes_sinasc = pd.concat([
# 	query_cnes_result('sus_sinasc', 10**6, 7777),
# 	query_cnes_result('sus_sinasc', 7777, 2600),
# 	query_cnes_result('sus_sinasc', 2600, 1100),
# 	query_cnes_result('sus_sinasc', 1100, 365),
# 	query_cnes_result('sus_sinasc', 365, 51),
# 	query_cnes_result('sus_sinasc', 51, 6),
# 	query_cnes_result('sus_sinasc', 6, 2),
# 	query_cnes_result('sus_sinasc', 2, 1, cnes_pref=True),
# 	query_cnes_result('sus_sinasc', 2, 1, cnes_pref=False),
# ])
path_cnes_sinasc = 'data/consult/compare/cnes_sinasc.csv'
# df_cnes_sinasc.to_csv(path_cnes_sinasc, index=False)
df_cnes_sinasc = pd.read_csv(
	path_cnes_sinasc,
	dtype={'CNES':str})
df_cnes_sinasc


	and CODESTAB like '2%'


,CNES,records
0,2079542,145885
1,3037274,115195
2,0002232,104344
3,0026794,103932
4,2323397,101349
...,...,...
8204,2810042,1
8205,2813483,1
8206,2818450,1
8207,2823330,1


In [13]:
set_cnes_sus = set(df_cnes['CNES'])

set_cnes_sih = set(df_cnes_sih['CNES'])
cnes_sus_sih = set_cnes_sus.intersection(set_cnes_sih)

set_cnes_sinasc = set(df_cnes_sinasc['CNES'])
cnes_sus_sinasc = set_cnes_sus.intersection(set_cnes_sinasc)

sus_sih = 15767633 # df_cnes_sih[df_cnes_sih['CNES'].isin(cnes_sus_sih)]['records'].sum()
sus_sinasc = 21196305 # df_cnes_sinasc[df_cnes_sinasc['CNES'].isin(cnes_sus_sih)]['records'].sum()

In [14]:
df_filter = pd.DataFrame([
	['SIH', sus_sih],
	['SINASC', sus_sinasc]
], columns=['database', 'records'])
fig = px.bar(df_filter, x='database', y='records')
fig.show()

## Comparação

In [15]:
df_filter = pd.DataFrame([
	['SIH', 'all', all_sih],
	['SINASC', 'all', all_sinasc],
	['SIH', 'filter', flt_sih],
	['SINASC', 'filter', flt_sinasc],
	['SIH', 'sus', sus_sih],
	['SINASC', 'sus', sus_sinasc],
], columns=['database', 'selection', 'records'])
fig = px.bar(df_filter, x='database', y='records', color='selection', barmode='overlay')
fig.show()

## CNES SUS em cada base

### SIH

In [16]:
df_cnes_sih[df_cnes_sih['CNES'].isin(set_cnes_sus)]

,CNES,records
1,3151794,75209
2,2323397,74417
3,2311682,64833
5,2020068,59610
6,6627595,58139
...,...,...
4144,7320175,1
4145,7603029,1
4146,7621442,1
4147,7704364,1


### SINASC

In [17]:
df_cnes_sinasc[df_cnes_sinasc['CNES'].isin(set_cnes_sus)]

,CNES,records
4,2323397,101349
5,2022621,99602
6,2311682,87673
7,2566168,87672
8,3151794,85933
...,...,...
8201,2806878,1
8202,2806924,1
8205,2813483,1
8206,2818450,1


## CNES no SIH

In [18]:
def read_cnes_query(path_query, prefix=1, init_two=False):
  with open(path_query) as f:
    query = f.read()
    rep_prefix = f'{prefix}%'
    rep_prefix = '2'+rep_prefix if init_two else rep_prefix
    query = query.replace('1%', rep_prefix)
    df = data_load.df_query(query)
  return df

In [19]:
def list_cnes(path_query):
  dfs = list()
  for i in tqdm(range(10)):
    if i == 2:
      for j in tqdm(range(10)):
        df_cnes_sus = read_cnes_query(path_query, j, i==2)
        dfs.append(df_cnes_sus)
    else:
      df_cnes_sus = read_cnes_query(path_query, i)
      dfs.append(df_cnes_sus)
  df_cnes_sus = pd.concat(dfs)
  return df_cnes_sus

In [20]:
path_query = 'queries/compare/cnes_sih.sql'
df_cnes_sih = list_cnes(path_query)
df_cnes_sih.to_csv('cnes_sih.csv.zip', index=False)
df_cnes_sih = pd.read_csv('cnes_sih.csv.zip', dtype={'CNES':str})
df_cnes_sih

100%|██████████| 10/10 [00:21<00:00,  2.10s/it]


,CNES
0,0000027
1,0000035
2,0000094
3,0000396
4,0000418
...,...
4472,9677011
4473,9680500
4474,9813349
4475,9917322


In [21]:
df_sinasc_cnes = df_cnes_sinasc[df_cnes_sinasc['CNES'].isin(set(df_cnes_sih['CNES']))]
df_sinasc_cnes

,CNES,records
2,0002232,104344
3,0026794,103932
4,2323397,101349
5,2022621,99602
6,2311682,87673
...,...,...
8162,2761157,1
8170,2772213,1
8179,2777819,1
8187,2789345,1


In [22]:
sih_sinasc = df_sinasc_cnes['records'].sum() # 22928708
sih_sinasc

22899941

In [23]:
df_filter = pd.DataFrame([
	['SIH', 'all', all_sih],
	['SINASC', 'all', all_sinasc],
	['SIH', 'filter', flt_sih],
	['SINASC', 'filter', flt_sinasc],
	['SIH', 'sus', sus_sih],
	['SINASC', 'sus', sih_sinasc],
], columns=['database', 'selection', 'records'])
fig = px.bar(df_filter, x='database', y='records', color='selection', barmode='overlay')
fig.show()